# Workflows

As noted in the article ["Building effective agents" by Anthropic](https://www.anthropic.com/research/building-effective-agents), workflows can be an effective way to use LLMs to complete complex tasks. Our experience building the first iteration of the "note formatting" feature, small LLMs are not capable of completing such complicated task in one step yet. Our goal in this exploration is to see if breaking down the task in multiple steps of one workflow can significantly improve realability of the system and accuracy.

### Design

1. Original note
2. (Assess note quality)
3. Make concise (input: original note + edited note)
4. Fix code blocks (input: original note + edited note) 
5. Fix math blocks (input: original note + edited note)
6. Fix media (input: original note + edited note)
7. Fix HTML formatting (input: original note + edited note)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import html
from jinja2 import Template

from anki_ai.domain.model import Deck, Note
from anki_ai.adapters.chat_completion import get_completion
from anki_ai.service_layer.services import (
    get_chat_completion,
    ChatCompletionsService,
    strip_html_tags,
    format_note,
)

In [3]:
deck = Deck()
deck.read_txt("../data/Selected Notes v8.txt", exclude_tags=["personal"])
print(f"The deck contains {len(deck)} notes.")

The deck contains 2854 notes.


Let's take a random sample of notes, to iterate more rapidly in our initial exploration. 

In [4]:
deck.shuffle()
sample = deck[:20]  # NOTE: this returs a list, should it instead return a Deck?

Now that we have a small dataset we can use, let's start working on creating those steps in the workflow. 

In [5]:
def add_html_tags(s: str) -> str:
    return html.escape(s).replace("\n", "<br>")


def remove_html_tags(s: str) -> str:
    return html.unescape(s).replace("<br>", "\n")

In [6]:
chat_completion = get_chat_completion()
chat = get_completion()

In [7]:
system_msg_tmpl = Template(r"""Your job is to optimize Anki notes, and particularly to make each note:
- Concise, simple, distinct
- Follow formatting rules
- Use valid Markdown syntax

You will be present with an existing note, including front, back, and tags. You must create a new note preserving its original meaning, and preserving any image, code block, and math block. 

### Formatting rules

The following rules apply to both front and back of each note.

Terminal commands must follow this format:
```bash
$ command <placeholder>
```

Code snippets must follow this format:
```language
code here
```

Name of programs, utilities, and tools like nvim, systemctl, pandas, grep, etc. must follow this format:
`nvim`, `systemctl`, `pandas`, `grep`

Keyboard keys and keymaps must follow this format:
`<C-aa>`, `x`, `J`, `gg`, `<S-p>`

In code blocks, use only the following placeholders: <file>, <path>, <link>, <command>.

Represent newlines with the `<br>` tag instead of `\n`. 

### Other rules

Always copy to the new note, without any modification, code bdlocks and images from the original note. 

Wrap back of a note within double quotes.

No explanations.

Return results using this JSON schema:
{
    "title": "Note",
    "type": "object",
    "properties": {
        "front": {"type": "string"},
        "back": {"type": "string"},
        "tags": {"type": "string"},
    },
    "required": ["front", "back", "tags"]
}

### Examples

Example 1: Code block
Input: Front: What command does extract files from a zip archive?
Back: ```bash
$ unzip <file>
Tags: ['linux']
```
Output: { "front": "Extract zip files", "back": "\"```bash<br>$ unzip <file><br>```\"", "tags": ['linux'] }

Example 2: Cloze completion
Input: Front: What type of memory do GPUs come equipped with?
* \{\{c1::Dynamic RAM (HBM)\}\}
* \{\{c2::Static RAM (L1 + L2 + Registers)\}\}
Back: 
Tags: ['recsys'] 
```
Output: { "front": "Type of memory on a GPU:<br>* \{\{c1::Dynamic RAM (HBM)\}\}<br>* \{\{c2::Static RAM (L1 + L2 + Registers)\}\}", "back": "\"\"", "tags": ['linux'] }

Example 3: Code block with placeholders
Input: Front: What command creates a soft link?
Back: ```bash
$ ln -s <file_name> <link_name>
```
Tags: ['linux']
Output: { "front": "Create soft link", "back": "\"```bash<br>$ ln -s <file> <link><br>```\"", "tags": ['linux'] }

Example 4: Code block and inline code block
Input: Front: In the `ln -s` command, what is the order of file name and link name?
Back: ```bash
$ ln -s <file_name> <link_name>
```
Tags: ['linux']
Output: { "front": "`ln -s` argument order", "back": "\"<file> then <link>\"", "tags": ['linux'] }

Example 5: Math
Input: Front: What is the range of the Leaky ReLU function?
Back: $ [ -0.01, + \infty ] $
Tags: ['dl']
Output: { "front": "Leaky ReLU range", "back": "\"$ [-0.01, +\infty] $\"", "tags": ['dl'] }

Example 6: Inline code block
Input: Front: What key returns the `^` in the shifted state?
Back: "`6`"
Tags: ['keyboard']
Output: { "front": "Keyboard key for `^` in shifted state", "back": "\"`6`\"", "tags": ['keyboard'] }


Input: {{ input_note }}
Output: """)

In [8]:
def format_note_v2(note: Note, chat: ChatCompletionsService = chat) -> Note:
    user_msg = (
        lambda note: f"""Front: {remove_html_tags(note.front)}\nBack: {remove_html_tags(note.back)}\nTags: {note.tags}\n"""
    )
    system_msg = lambda note: system_msg_tmpl.render(input_note=note)
    messages = system_msg(user_msg(note))
    
    chat_response = chat.create(
        model="meta-llama/Meta-Llama-3.1-8B-Instruct",
        prompt=messages,
        temperature=0,
        max_tokens=500,
        extra_body={
            "guided_json": Note.model_json_schema(),
        },
    )

    json_data = chat_response.choices[0].text
    new_note = Note.model_validate_json(json_data)
    return new_note

In [9]:
import re

def remove_alt_tags(note: Note) -> Note:
    note.front = re.sub(r'<img\s+alt="+[^"]*"+', '<img alt=""', note.front)
    note.back = re.sub(r'<img\s+alt="+[^"]*"+', '<img alt=""', note.back)
    return note

In [10]:
def pipe(note: Note, chat: ChatCompletionsService) -> Note:
    note = format_note_v2(note, chat)
    note = remove_alt_tags(note)
    return note

In [11]:
def compare_notes(original: Note, baseline: Note, challenger: Note) -> None:
    print(f"Front: {note.tags} {remove_html_tags(note.front)}\nBack: {remove_html_tags(note.back)}\n")

    print(f"Front: {baseline.tags} {baseline.front}\nBack: {baseline.back}\n")
    print(f"Front: {challenger.tags} {challenger.front}\nBack: {challenger.back}\n")      
    print("#####################\n")

In [12]:
for note in sample:
    baseline = format_note(note, chat_completion)
    challenger = pipe(note, chat)
    compare_notes(note, baseline, challenger)

Front: ['nvim'] Motion to move the cursor [count] WORDS forward
Back: "`[count]W`

<img src=""HYaJn.gif"">"

Front: ['nvim'] Motion to move cursor WORDS forward
Back: `[count]W`

Front: ['nvim'] Motion to move the cursor [count] WORDS forward
Back: "`[count]W`<br><img src="HYaJn.gif">"

#####################

Front: ['cycling'] "What type of tyres can a hooked rim fit?

<img src=""hooked-rim-fast-fwd-1.jpeg"">"
Back: Clincher and (if specified) tubeless-ready tyres

Front: ['cycling'] What type of tyres can a hooked rim fit?
Back: Clincher and (if specified) tubeless-ready tyres

Front: ['cycling'] Hooked rim tyre type
Back: "Clincher and (if specified) tubeless-ready tyres"

#####################

Front: ['linux'] What command line tool controls the Bluetooth  connections?
Back: `bluetoothctl`

Front: ['linux'] Bluetooth connection tool
Back: ```
bluetoothctl
```

Front: ['linux'] Bluetooth connection tool
Back: "`bluetoothctl`"

#####################

Front: ['keyboard'] What key ret

We are making the LLM process the strings without HTML special characters and tags. The reason for doing this is that we are removing some of the complexity, and making human eval easier.

Another exception is that we make the LLM generate newlines using the HTML tag `<br>`, because we noticed it confuses less the model. When using `\n`, the LLM sometimes starts generating random sequences of symbols.

This new approach seems to lead to better results. There are still minor issues with the recommended changes, that we can try to fix with specialized prompts.

#### Not handled yet:
- [ ] Notes where front or back card contain only an image
- [x] Cloze completions
- [ ] Bullet points
- [x] Remove content from `alt` argument in `<img>` tag
- [ ] Code example in front/back that we want to preserve

#### Other things to try:
- [ ] Show note to LLM as it will be shown to user: `[{{ Tags}}] {{ Front }}\n {{ Back }} `
- [x] Do not generate node ID and tags
- [ ] Wrap also front content within double quotes
- [ ] Different diff formats (`whole` --> `diff`). See [Aider's research](https://aider.chat/docs/more/edit-formats.html) on this topic
- [x] Create simple utility function to compare recommendations from different workflows

In [17]:
!head ../data/Selected\ Notes\ v8.txt -n 25

#separator:tab
#html:true
#guid column:1
#notetype column:2
#deck column:3
#tags column:9
D?H@y-%%r	KaTeX and Markdown Basic (Color)	Default	"<img src=""paste-d0ff77498ff8dde85ba00ae8b7c4bb6032d8483d.jpg"">"	Headboard				english
IjfKk}wnb@	KaTeX and Markdown Basic (Color)	Default	"<img src=""paste-334a3566ffa4cab66033c10810e8d06af8fda194.jpg"">"	Towel				english
"G1Z_~#;mLc"	KaTeX and Markdown Basic (Color)	Default	"<img src=""paste-d9689dc830d3f333e81b9b7058d5b25517064954.jpg"">"	Jug				english
Azd65{j+,q	KaTeX and Markdown Basic (Color)	Default	Command to create a soft link	```bash<br>$ ln -s &lt;file&gt; &lt;link&gt;<br>```				linux
BGL!8$wV<W	KaTeX and Markdown Basic (Color)	Default	In the `ln -s` command, what is the order of file name and link name?	```bash<br>$ ln -s &lt;file_name&gt; &lt;link_name&gt;<br>```				linux
be:y>MF$Ae	KaTeX and Markdown Basic (Color)	Default	In the `zip` command, what is the option to specify the destination?	"```bash<br>$ unzip &lt;file&gt; -d &lt;pa

In [16]:
deck.get("D?H@y-%%r")

[Note(guid='D?H@y-%%r', front='"<img src=""paste-d0ff77498ff8dde85ba00ae8b7c4bb6032d8483d.jpg"">"', back='Headboard', tags=['english'], notetype='KaTeX and Markdown Basic (Color)', deck_name='Default')]